In [10]:
# Model 
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination 
from dotenv import load_dotenv
from autogen_agentchat.base import TaskResult
from autogen_agentchat.ui import Console
import os
load_dotenv()

model_client = OpenAIChatCompletionClient(model="gpt-4o",api_key=os.getenv("OPENAI_API_KEY"))


In [ ]:

from autogen_agentchat.conditions import TextMentionTermination
import json
TERMINATION_WORD = 'stop'




def save_state(agent,filename):
    """
    Save the state of the agent to a file.
    """
    state = agent.save_state()
    with open(filename, "w") as f:
        json.dump(state, f)

def load_state(agent, filename):
    """
    Load the state of the agent from a file.
    """
    with open(filename, "r") as f:
        state = json.load(f)
    agent.load_state(state)

#def get_termination_condition():
#    """
#    Get the termination condition for the agent.
#    """
#    text_mention_termination = TextMentionTermination(TERMINATION_WORD)
#    return text_mention_termination

# Create Termination Condition
from autogen_agentchat.conditions import MaxMessageTermination
def get_termination_condition():
    return MaxMessageTermination(max_messages=5)

In [12]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat


# Cell 4: Create Agents
planner_agent = AssistantAgent(
    name="travel_planner",
    model_client=model_client,
    system_message="""You are a professional travel planner. Your role is to:
    - Create detailed day-by-day itineraries
    - Suggest optimal timing and logistics
    - Recommend transportation between locations
    - Provide practical travel advice
    - Consider budget constraints and preferences"""
)

research_agent = AssistantAgent(
    name="travel_researcher",
    model_client=model_client,
    system_message="""You are a travel research specialist. Your role is to:
    - Find information about destinations and attractions
    - Research accommodation options
    - Provide cultural and historical context
    - Share local tips and recommendations
    - Research weather, seasons, and best times to visit"""
)



# Cell 6: Create Team
team = RoundRobinGroupChat(
    participants=[planner_agent, research_agent],
    termination_condition=get_termination_condition()
)

In [13]:
async def run_travel_planning(user_request):
    """Run the travel planning team with a user request"""
    initial_message = f"""Travel Planning Request: {user_request}
    
    Please work together as a team to help plan this trip. 
    - Travel Researcher: Start by providing destination information, attractions, and recommendations
    - Travel Planner: Then create a detailed itinerary based on the research
    
    Collaborate to provide the best possible travel plan."""
    
    try:
        result = await team.run(task=initial_message)
        return result
    except Exception as e:
        print(f"Error during travel planning: {e}")
        return None
    finally:
        # Clean up
        await model_client.close()


In [14]:
async def plan_trip(destination, duration, budget=None, interests=None):
    """Wrapper function to plan a trip"""
    user_request = f"""
    Destination: {destination}
    Duration: {duration}
    Budget: {budget or "Flexible"}
    Interests: {interests or "General sightseeing, culture, food"}
    
    Please help me plan a comprehensive trip including:
    1. Must-visit attractions and activities
    2. Day-by-day itinerary
    3. Accommodation recommendations
    4. Transportation suggestions
    5. Local tips and cultural insights
    """
    
    return await run_travel_planning(user_request)

In [15]:
# CHANGED: Use team.run_stream() instead of result.run_stream()
user_request = """
Destination: Tokyo, Japan
Duration: 5 days
Budget: $2000
Interests: Technology, culture, food

Please help me plan a comprehensive trip including:
1. Must-visit attractions and activities
2. Day-by-day itinerary
3. Accommodation recommendations
4. Transportation suggestions
5. Local tips and cultural insights
"""

await Console(team.run_stream(task=f"""Travel Planning Request: {user_request}

Please work together as a team to help plan this trip. 
- Travel Researcher: Start by providing destination information, attractions, and recommendations
- Travel Planner: Then create a detailed itinerary based on the research

Collaborate to provide the best possible travel plan."""))

---------- TextMessage (user) ----------
Travel Planning Request: 
Destination: Tokyo, Japan
Duration: 5 days
Budget: $2000
Interests: Technology, culture, food

Please help me plan a comprehensive trip including:
1. Must-visit attractions and activities
2. Day-by-day itinerary
3. Accommodation recommendations
4. Transportation suggestions
5. Local tips and cultural insights


Please work together as a team to help plan this trip. 
- Travel Researcher: Start by providing destination information, attractions, and recommendations
- Travel Planner: Then create a detailed itinerary based on the research

Collaborate to provide the best possible travel plan.


---------- TextMessage (travel_planner) ----------
**Travel Researcher**

**Tokyo, Japan Overview:**

Tokyo, a city where tradition meets futuristic technology, offers a blend of cultural and modern experiences. With your interests in technology, culture, and food, Tokyo has numerous attractions that will cater to your preferences.

**Must-Visit Attractions and Activities:**

1. **Technology:**
   - **Akihabara:** Known as the hub for electronics and otaku culture, visit stores like Yodobashi Camera and manga cafes.
   - **TeamLab Planets Tokyo:** An immersive digital art museum located in Toyosu.
   - **Miraikan – The National Museum of Emerging Science and Innovation:** Explore cutting-edge science developments.

2. **Culture:**
   - **Senso-ji Temple, Asakusa:** Tokyo's oldest temple with a bustling shopping street, Nakamise Dori.
   - **Meiji Shrine:** A Shinto shrine dedicated to Emperor Meiji and Empress Shoken.
   - **Tokyo National Museum:** Offers extensive collections of trad

TaskResult(messages=[TextMessage(id='d93d9e04-bc5c-4dbb-8632-128b53b63038', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 9, 0, 59, 27, 616792, tzinfo=datetime.timezone.utc), content='Travel Planning Request: \nDestination: Tokyo, Japan\nDuration: 5 days\nBudget: $2000\nInterests: Technology, culture, food\n\nPlease help me plan a comprehensive trip including:\n1. Must-visit attractions and activities\n2. Day-by-day itinerary\n3. Accommodation recommendations\n4. Transportation suggestions\n5. Local tips and cultural insights\n\n\nPlease work together as a team to help plan this trip. \n- Travel Researcher: Start by providing destination information, attractions, and recommendations\n- Travel Planner: Then create a detailed itinerary based on the research\n\nCollaborate to provide the best possible travel plan.', type='TextMessage'), TextMessage(id='250b290b-39b2-46bd-9620-18e01fed10ea', source='travel_planner', models_usage=RequestUsage(prompt_to